In [4]:
### Save API Token, if needed

%set_env QXToken=c8d820fbfd256c7f80878c331c19ef7a79c7fd5b2c5551813745c137482901e1fc2035b303fd01a65f6c6573ec62654ab509489703235c5298bde34febc8fd1b
# Make sure there is no space between the equal sign
# and the beginning of your token

env: QXToken=c8d820fbfd256c7f80878c331c19ef7a79c7fd5b2c5551813745c137482901e1fc2035b303fd01a65f6c6573ec62654ab509489703235c5298bde34febc8fd1b


In [5]:
# qc-grader should be 0.18.12 (or higher)
import qc_grader

qc_grader.__version__

'0.18.12'

In [6]:
# Import all in one cell

import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer
import warnings

from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import TwoLocal, EfficientSU2
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session, Options
from qiskit_serverless import QiskitFunction, save_result, get_arguments, save_result, distribute_task, distribute_qiskit_function, IBMServerlessClient, QiskitFunction
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_transpiler_service.transpiler_service import TranspilerService
from qiskit_aer import AerSimulator

In [7]:
# Import for grader
from qc_grader.challenges.iqc_2024 import grade_lab3_qs_ex1, grade_lab3_qs_ex2

In [8]:
# If needed setup your QiskitRuntimeService

QiskitRuntimeService.save_account(
    channel="ibm_quantum",
    token="c8d820fbfd256c7f80878c331c19ef7a79c7fd5b2c5551813745c137482901e1fc2035b303fd01a65f6c6573ec62654ab509489703235c5298bde34febc8fd1b",
    set_as_default=True,
    # Use `overwrite=True` if you're updating your token.
    overwrite=True,
)

service = QiskitRuntimeService(channel="ibm_quantum")
 
# Specify a system to use for transpilation
real_backend = service.backend("ibm_brisbane")

In [9]:
# Qiskit Pattern Step 1: Map quantum circuits and operators (Define Ansatz and operators)
num_qubits = 3
rotation_blocks = ['ry','rz']
entanglement_blocks = 'cz'
entanglement = 'full'
# Define Ansatz
ansatz = TwoLocal(num_qubits=num_qubits,rotation_blocks=rotation_blocks,entanglement_blocks=entanglement_blocks,entanglement=entanglement,reps=1,insert_barriers=True)


# Define parameters
num_params = ansatz.num_parameters

# Qiskit Pattern Step 2: Optimize the circuit for quantum execution
optimization_level = 2
pm = generate_preset_pass_manager(backend=real_backend, optimization_level=optimization_level)
isa_circuit = pm.run(ansatz)

# Define Hamiltonian for VQE
pauli_op = SparsePauliOp(['ZII', 'IZI', 'IIZ'])
hamiltonian_isa = pauli_op.apply_layout(layout=isa_circuit.layout)

In [10]:
# Setup Qiskit Serverless Client and Qiskit Runtime client
client = IBMServerlessClient("c8d820fbfd256c7f80878c331c19ef7a79c7fd5b2c5551813745c137482901e1fc2035b303fd01a65f6c6573ec62654ab509489703235c5298bde34febc8fd1b") # Add in your IBM Quantum Token to QiskitServerless Client

# For the challenge, we will be using QiskitRuntime Local testing mode. Change to True only if you wish to use real backend.
USE_RUNTIME_SERVICE = False

if USE_RUNTIME_SERVICE:
    service = QiskitRuntimeService(
        channel='ibm_quantum', 
        verify=False
    )
else:
    service = None

In [11]:
# Define the Qiskit Function
if USE_RUNTIME_SERVICE:
    function = QiskitFunction(title= "vqe", entrypoint="vqe.py", working_dir="C:\\Users\\kldab\\Desktop\\lab3\\vqe")
else:
    function = QiskitFunction(title= "vqe" , entrypoint="vqe.py", working_dir="C:\\Users\\kldab\\Desktop\\lab3\\vqe",  dependencies=["qiskit_aer"])

In [12]:
# Upload the Qiskit Function using IBMServerlessClient
client.upload(function)

'vqe'

In [13]:
# Define input_arguments
input_arguments = {
    "ansatz":isa_circuit, # Replace with your transpiled ansatz
    "operator":hamiltonian_isa, # Replace with the hamiltonian operator
    "method":"COBYLA",# Using COBYLA method for the optimizer
    "service":service # Add your code here
}

# Qiskit Pattern Step 3: Run the payload on backend
job = client.run("vqe", arguments=input_arguments)

In [14]:
# Submit your answer using following code

grade_lab3_qs_ex1(function, input_arguments, job)
# Expected result type: QiskitFunction, dict, Job

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [15]:
# Return jobid
job

<Job | 806a511c-553f-41ae-8930-e5acc321ffc9>

In [16]:
# Check job completion status
job.status()

'QUEUED'

In [17]:
# Monitor log
logs = job.logs()

for log in logs.splitlines():
    print(log)

No logs yet.


In [ ]:
# Return result from QiskitFunction job
job.result()

In [ ]:
# Qiskit Pattern Step 4: Postprocess and analyze the Estimator V2 results
result = job.result()

fig, ax = plt.subplots()
plt.plot(range(result["iters"]), result["cost_history"])
plt.xlabel("Energy")
plt.ylabel("Cost")
plt.draw()

In [20]:
# Setup 3 circuits with Efficient SU2
num_qubits = [41, 51, 61]
circuits = [EfficientSU2(nq, su2_gates=["rz","ry"], entanglement="circular", reps=1).decompose() for nq in num_qubits]

In [21]:
# Setup Qiskit Runtime Service backend

QiskitRuntimeService.save_account(
    channel="ibm_quantum",
    token='c8d820fbfd256c7f80878c331c19ef7a79c7fd5b2c5551813745c137482901e1fc2035b303fd01a65f6c6573ec62654ab509489703235c5298bde34febc8fd1b',
    set_as_default=True,
    # Use 'overwrite=True' if you're updating your token.
    overwrite=True,
)

service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibm_brisbane")

In [22]:
# Define Configs
optimization_levels = [1,2,3]
pass_managers = [{'pass_manager': generate_preset_pass_manager(optimization_level=level, backend=backend), 'optimization_level': level} for level in optimization_levels]

transpiler_services = [ 
        {'service': TranspilerService(backend_name="ibm_brisbane",ai="false",optimization_level=3), 'ai': False, 'optimization_level': 3},
        {'service': TranspilerService(backend_name="ibm_brisbane",ai="true",optimization_level=3), 'ai': True, 'optimization_level': 3}
    ]

configs = pass_managers + transpiler_services

In [23]:
# Local transpilation setup
def transpile_parallel_local(circuit: QuantumCircuit, config):
    """Transpilation for an abstract circuit into an ISA circuit for a given config."""
    transpiled_circuit = config.run(circuit)
    return transpiled_circuit

In [ ]:

# Run local transpilation
warnings.filterwarnings("ignore")

start = timer()

# Run transpilations locally for baseline
results = []
for circuit in circuits:
    for config in configs:
        if 'pass_manager' in config:
            results.append(transpile_parallel_local(circuit, config['pass_manager']))
        else:
            results.append(transpile_parallel_local(circuit, config['service']))
            
end = timer()

# Record local execution time
execution_time_local = end - start
print("Execution time locally: ", execution_time_local)

INFO:qiskit_transpiler_service.transpiler_service:Requesting transpile to the service
INFO:qiskit_transpiler_service.transpiler_service:Qiskit transpiler service returned a result
INFO:qiskit_transpiler_service.transpiler_service:Requesting transpile to the service
INFO:qiskit_transpiler_service.transpiler_service:Qiskit transpiler service returned a result
INFO:qiskit_transpiler_service.transpiler_service:Requesting transpile to the service
INFO:qiskit_transpiler_service.transpiler_service:Qiskit transpiler service returned a result
INFO:qiskit_transpiler_service.transpiler_service:Requesting transpile to the service
INFO:qiskit_transpiler_service.transpiler_service:Qiskit transpiler service returned a result
INFO:qiskit_transpiler_service.transpiler_service:Requesting transpile to the service
INFO:qiskit_transpiler_service.transpiler_service:Qiskit transpiler service returned a result
INFO:qiskit_transpiler_service.transpiler_service:Requesting transpile to the service
INFO:qiskit_tr

Execution time locally:  575.5635851999978


In [24]:

# Authenticate to the remote cluster and submit the pattern for remote execution if not done in previous exercise
serverless = IBMServerlessClient("c8d820fbfd256c7f80878c331c19ef7a79c7fd5b2c5551813745c137482901e1fc2035b303fd01a65f6c6573ec62654ab509489703235c5298bde34febc8fd1b")

In [32]:
transpile_parallel_function = QiskitFunction(
    title="transpile_parallel",
    entrypoint="transpile_parallel.py",
    working_dir="C:\\Users\\kldab\\Desktop\\lab3\\transpile_parallel",
    dependencies=["qiskit-transpiler-service"]
)

In [33]:
serverless.upload(transpile_parallel_function)

'transpile_parallel'

In [34]:
serverless.list()

[QiskitFunction(vqe), QiskitFunction(transpile_parallel)]

In [35]:
# Fetch the specific function titled "transpile_parallel"
transpile_parallel_serverless = serverless.get('transpile_parallel')

In [36]:
# Run the "transpile_parallel" function in the serverless environment
job = transpile_parallel_serverless.run()

In [37]:
# Submit your answer using following code

grade_lab3_qs_ex2(optimization_levels, transpiler_services, transpile_parallel_function, transpile_parallel_serverless, job)
# Expected result type: list, list, QiskitFunction, QiskitFunction, Job

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.
